# Floor Plan Generator

This notebook provides an interface to generate floor plans. You can input a text prompt, choose the number of images to generate, select the inference steps, and pick the scheduler.

### Imports

In [2]:
from diffusers import (DDIMScheduler, PNDMScheduler, EulerDiscreteScheduler, 
                       DPMSolverMultistepScheduler, HeunDiscreteScheduler, 
                       EulerAncestralDiscreteScheduler)
from diffusers import AutoPipelineForText2Image
import ipywidgets as widgets
import torch
from ipywidgets import widgets, VBox, Output, Button
from IPython.display import display

### Define necessary functions

In [3]:
def generate_images(prompt, num_images, scheduler, steps):
    """
    Generates images based on the given prompt using a text-to-image model.

    Args:
        prompt (str): The text prompt for generating the images.
        num_images (int): The number of images to generate.
        scheduler (str): The type of scheduler to use for the diffusion model.
        steps (int): The number of inference steps to perform.

    Returns:
        list: A list of generated images.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    pipeline = AutoPipelineForText2Image.from_pretrained("runwayml/stable-diffusion-v1-5").to(device)

    # Load the LoRA weights for the model
    pipeline.load_lora_weights("C:/Users/see/Documents/GitHub/Diffusion-Models-for-floor-plan-drafting/Checkpoints_L1/checkpoint-250", weight_name="pytorch_lora_weights.safetensors")

    # Set scheduler 
    if scheduler == "DDIM":
        pipeline.scheduler = DDIMScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")
    elif scheduler == "PNDM":
        pipeline.scheduler = PNDMScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")
    elif scheduler == "EulerDiscrete":
        pipeline.scheduler = EulerDiscreteScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")
    elif scheduler == "DPMSolverMultistep":
        pipeline.scheduler = DPMSolverMultistepScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")
    elif scheduler == "HeunDiscrete":
        pipeline.scheduler = HeunDiscreteScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")
    elif scheduler == "EulerAncestralDiscrete":
        pipeline.scheduler = EulerAncestralDiscreteScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")

    # Generate images based on the prompt
    images = []
    for _ in range(num_images):
        image = pipeline(prompt, num_inference_steps=steps).images[0]
        images.append(image)
    return images

def display_images(prompt, num_images, steps, scheduler):
    """
    Display a series of generated images.

    Args:
        prompt (str): The prompt for generating the images.
        num_images (int): The number of images to generate and display.
        steps (int): The number of steps to perform during image generation.
        scheduler: The scheduler used for generating the images.
    """
    images = generate_images(prompt, num_images, scheduler, steps)
    for idx, img in enumerate(images):
        display(img)
        

### Create Widgets and Button

In [4]:
# Create widgets
prompt_widget = widgets.Textarea(value='', placeholder='Enter your prompt...', description='Prompt:')
num_images_widget = widgets.IntSlider(value=1, min=1, max=4, step=1, description='Number of Images:')
steps_widget = widgets.IntSlider(value=50, min=1, max=100, step=1, description='Inference Steps:')
scheduler_widget = widgets.Dropdown(options=["DDIM", "PNDM", "EulerDiscrete", "DPMSolverMultistep", "HeunDiscrete", "EulerAncestralDiscrete"], value='DDIM', description='Scheduler:')
generate_button = Button(description="Generate")
output = Output()

# Define the function to be triggered by the button
def on_generate_button_clicked(b):
    """
    Event handler for the generate button click event.

    This function clears the output widget and displays the generated images based on the user's input.

    Parameters:
    - b: The button object that triggered the event.
    """
    with output:
        output.clear_output()
        display_images(prompt_widget.value, num_images_widget.value, steps_widget.value, scheduler_widget.value)

generate_button.on_click(on_generate_button_clicked)

### Display interface

In [5]:
# Display the widgets and button
display(VBox([prompt_widget, num_images_widget, steps_widget, scheduler_widget, generate_button, output]))